In [24]:
import os
import pandas as pd
import numpy as np
import pandas_datareader
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
import yfinance as yf

def stockData(sList, stDate, endDate):
    """Take list of stocks and pull data for that stock and create technical indicator variables
    sList requires list of stocks, dates should be entered in format 'YYYY-MM-DD' """
   
    data = []
    for element in sList:
        try:
            temp = pdr.get_data_yahoo(element, start = stDate, end = endDate)
            #add an indentifier
            temp['Symbol'] = element
            temp['Close'] = temp['Close']
        except: 
            #If stock symbol cannot be found make it $1
            print (element)
            temp['Symbol'] = element
            temp['Close'] = 1
        else:
        #add data to list 
            data.append(temp)
        #convert to dataframe    
        df = pd.concat(data)
        #drop rows for which we will not have all data points calculated
        df = df.dropna()
        #dates above what user entered
        df = df.loc[stDate : endDate]
        #order and limit columns to those needed
        df = df.loc[:, ['Symbol', 'Close']]
        df['Day'] = pd.to_datetime(df.index)

    
    return(df)


In [25]:
#set working directory 
os.chdir("/Users/bkrei/Desktop/Bk's Stuff Desktop/School/Github NEW PATH/Portfolio-Challenge")

In [42]:
#Read in individual portfolios
portfolios = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/New%20Baseline.csv")

In [28]:
port1 = portfolios.loc[portfolios['Port_Num'] == 1]

In [43]:
port2 = portfolios.loc[portfolios['Port_Num'] == 2]

In [30]:
#List of all ticker symbols
symbols1 = port1['Symbol'].drop_duplicates()
symbols1.head(10)

0     AAPL
1    AGTHX
2     AMZN
3      CHE
4     DOCU
5    GSGRX
6     ICLN
7    OEGAX
8     PBFS
9      QQQ
Name: Symbol, dtype: object

In [44]:
#List of all ticker symbols
symbols2 = port2['Symbol'].drop_duplicates()
symbols2.head(10)

283     AAPL
284      IBM
285      PIR
286      WMT
287    GOOGL
288     DOCU
289      GCO
290     ATLC
291      XOM
292      FCA
Name: Symbol, dtype: object

In [39]:
symbols1.to_csv("symbols port1.csv", header = False)
symbols2.to_csv("symbols port2.csv", header = False)

In [40]:
#Prices Port 1
#Pass the ticker list to the function, print invalid stock symbols and create dataframe with intial prices
beg_prices1 = stockData(list(symbols1), '2019-09-06', '2019-09-06')
beg_prices1.shape

CASHPIO


(160, 3)

In [56]:
beg_prices1.to_csv("prices_port1.csv", header = True)

In [35]:
#Error handling port 1
#er1 = stockData(['SNAP'], '2019-09-06', '2019-09-06')

In [45]:
#Prices Port 2
#Pass the ticker list to the function, print invalid stock symbols and create dataframe with intial prices

beg_prices2 = stockData(list(symbols2), '2019-09-27', '2019-09-27')
beg_prices2.shape

(104, 3)

In [8]:
#Pull SNAP which for some reason is the only stock with an issue.
#er2 = stockData(['SNAP'], '2019-09-27', '2019-09-27')

In [55]:
beg_prices2.to_csv("prices_port2.csv", header = True)

In [52]:
port1.head()

,Port_Num,New Cost Basis,Name,Symbol,% of Portfolio,Cost Basis
0,1,194413.7192,Adrienne Villano,AAPL,0.07,14000.0
1,1,194413.7192,Adrienne Villano,AGTHX,0.11,22000.0
2,1,194413.7192,Adrienne Villano,AMZN,0.07,14000.0
3,1,194413.7192,Adrienne Villano,CHE,0.18,36000.0
4,1,194413.7192,Adrienne Villano,DOCU,0.05,10000.0


In [9]:
#Combine the main and SNAP dataframes
#frames = [beg_prices, er]
#beg_prices = pd.concat(frames)

In [58]:
beg_prices1 = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/prices_port1.csv")


In [61]:
combined1 = pd.merge(port1, beg_prices1, how = 'left', on = "Symbol", validate="many_to_one")

In [62]:
combined1.head(5)

,Port_Num,New Cost Basis,Name,Symbol,% of Portfolio,Cost Basis,Close,Day
0,1,194413.7192,Adrienne Villano,AAPL,0.07,14000.0,213.259995,2019-09-06
1,1,194413.7192,Adrienne Villano,AGTHX,0.11,22000.0,50.000000,2019-09-06
2,1,194413.7192,Adrienne Villano,AMZN,0.07,14000.0,1833.510010,2019-09-06
3,1,194413.7192,Adrienne Villano,CHE,0.18,36000.0,439.820007,2019-09-06
4,1,194413.7192,Adrienne Villano,DOCU,0.05,10000.0,56.270000,2019-09-06


In [63]:
#Calc # of shares purchased
combined1["# of Shares"] = combined1["Cost Basis"]/ combined1["Close"]
#Convert names to proper case
#combined['Name'] = list(map(lambda x: x.title(), combined['Name']))
#combined.head()

In [64]:
beg_prices2 = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/prices_port2.csv")

In [65]:
combined2 = pd.merge(port2, beg_prices2, how = 'left', on = "Symbol", validate="many_to_one")

In [66]:
combined2.head(5)

,Port_Num,New Cost Basis,Name,Symbol,% of Portfolio,Cost Basis,Date,Close,Day
0,2,204011.836,Marie Barkamian,AAPL,0.1,20401.1836,2019-09-27,218.820007,2019-09-27
1,2,204011.836,Marie Barkamian,IBM,0.1,20401.1836,2019-09-27,143.240005,2019-09-27
2,2,204011.836,Marie Barkamian,PIR,0.1,20401.1836,2019-09-27,10.870000,2019-09-27
3,2,204011.836,Marie Barkamian,WMT,0.1,20401.1836,2019-09-27,118.449997,2019-09-27
4,2,204011.836,Marie Barkamian,GOOGL,0.2,40802.3672,2019-09-27,1225.949951,2019-09-27


In [67]:
#Calc # of shares purchased
combined2["# of Shares"] = combined2["Cost Basis"]/ combined2["Close"]
#Convert names to proper case
#combined['Name'] = list(map(lambda x: x.title(), combined['Name']))
#combined.head()

In [73]:
combined2 = combined2.drop(['Date'], axis=1)

In [74]:
#Combine the main and SNAP dataframes
two_frames = [combined1, combined2]
full_combined = pd.concat(two_frames)

In [91]:
full_combined = full_combined[full_combined.Name != 'Emmalie Giaimo']
full_combined['initial_basis'] = 200000

In [92]:
full_combined.to_csv("full_combine.csv")

In [93]:
new_base = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/full_combine.csv")

In [94]:
#List of all ticker symbols
symbols = new_base['Symbol'].drop_duplicates()
symbols.head(10)

0     AAPL
1    AGTHX
2     AMZN
3      CHE
4     DOCU
5    GSGRX
6     ICLN
7    OEGAX
8     PBFS
9      QQQ
Name: Symbol, dtype: object

# Simple Comparison (Initial vs Current Value) START HERE TO UPDATE

In [81]:
#Select date to compare to initial prices
cur_prices = stockData(list(symbols), '2019-10-03', '2019-10-03')

In [82]:
cur_prices.sort_values("Symbol", inplace = True)
cur_prices.drop_duplicates(subset = "Symbol", inplace = True)

In [83]:
cur_prices.to_csv("cur_prices.csv")

In [25]:
#cur_prices= pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/cur_prices.csv")
#cur_prices.head()


,Date,Symbol,Close,Day
0,2019-09-27,AAL,27.010000,2019-09-27
1,2019-09-27,AAPL,218.820007,2019-09-27
2,2019-09-27,ACB,4.620000,2019-09-27
3,2019-09-27,ACER,3.300000,2019-09-27
4,2019-09-27,ACST,1.960000,2019-09-27


In [95]:
detail = pd.merge(new_base, cur_prices, how = 'left', on = "Symbol", validate="many_to_one")

In [96]:
detail.head()

,Unnamed: 0,Port_Num,New Cost Basis,Name,Symbol,% of Portfolio,Cost Basis,Close_x,Day_x,# of Shares,initial_basis,Close_y,Day_y
0,0,1,194413.7192,Adrienne Villano,AAPL,0.07,14000.0,213.259995,2019-09-06 00:00:00,65.647568,200000,220.820007,2019-10-03
1,1,1,194413.7192,Adrienne Villano,AGTHX,0.11,22000.0,50.000000,2019-09-06 00:00:00,440.000000,200000,48.520000,2019-10-03
2,2,1,194413.7192,Adrienne Villano,AMZN,0.07,14000.0,1833.510010,2019-09-06 00:00:00,7.635628,200000,1724.420044,2019-10-03
3,3,1,194413.7192,Adrienne Villano,CHE,0.18,36000.0,439.820007,2019-09-06 00:00:00,81.851665,200000,414.380005,2019-10-03
4,4,1,194413.7192,Adrienne Villano,DOCU,0.05,10000.0,56.270000,2019-09-06 00:00:00,177.714589,200000,62.209999,2019-10-03


In [97]:
detail['Current_Value'] = detail['Close_y'] * detail['# of Shares']
detail['Perc_Return'] = detail['Close_y']/detail['Close_x']  -1
detail.head(5)

,Unnamed: 0,Port_Num,New Cost Basis,Name,Symbol,% of Portfolio,Cost Basis,Close_x,Day_x,# of Shares,initial_basis,Close_y,Day_y,Current_Value,Perc_Return
0,0,1,194413.7192,Adrienne Villano,AAPL,0.07,14000.0,213.259995,2019-09-06 00:00:00,65.647568,200000,220.820007,2019-10-03,14496.296456,0.035450
1,1,1,194413.7192,Adrienne Villano,AGTHX,0.11,22000.0,50.000000,2019-09-06 00:00:00,440.000000,200000,48.520000,2019-10-03,21348.800201,-0.029600
2,2,1,194413.7192,Adrienne Villano,AMZN,0.07,14000.0,1833.510010,2019-09-06 00:00:00,7.635628,200000,1724.420044,2019-10-03,13167.029624,-0.059498
3,3,1,194413.7192,Adrienne Villano,CHE,0.18,36000.0,439.820007,2019-09-06 00:00:00,81.851665,200000,414.380005,2019-10-03,33917.693437,-0.057842
4,4,1,194413.7192,Adrienne Villano,DOCU,0.05,10000.0,56.270000,2019-09-06 00:00:00,177.714589,200000,62.209999,2019-10-03,11055.624414,0.105562


In [87]:
detail.to_csv("pt2-detail.csv")

### Individual Performance

In [104]:
ranking = detail.groupby("Name")['Current_Value'].sum().reset_index().sort_values("Current_Value", ascending=False)

In [107]:
ranking['Perc_Return'] = ranking['Current_Value']/200000 -1
ranking['Rank'] = ranking['Perc_Return'].rank(ascending=False)
ranking.head(50)

,Name,Current_Value,Perc_Return,Rank
12,Colton Young,218311.322514,0.091557,1.0
25,Joelean Mcbride,210732.212648,0.053661,2.0
26,Joseph Lindner,204633.016109,0.023165,3.0
46,Robert Kovacs,204119.000226,0.020595,4.0
28,Kelly Nicholson,203007.946389,0.015040,5.0
5,Anil Mulchandani,199703.494276,-0.001483,6.0
21,Gregory Hanselman,199099.906967,-0.004500,7.0
9,Bradley Bonfante,198481.134375,-0.007594,8.0
36,Marie Barkamian,198102.593247,-0.009487,9.0
20,Estelle Genest,198089.425713,-0.009553,10.0


In [108]:
ranking.to_csv("individual rankings.csv")

### Best Individual Stock Returns

In [109]:
best_stocks = detail.drop_duplicates(['Symbol']).reset_index().sort_values(['Perc_Return'], ascending=False)

In [110]:
best_stocks = best_stocks.loc[:, ['Symbol', 'Perc_Return']]
best_stocks.head(10)

,Symbol,Perc_Return
63,PIR,0.514610
111,HOME,0.277624
167,DRIP,0.172521
96,VHC,0.170094
153,RVNC,0.142594
113,RH,0.137050
171,WORK,0.130100
4,DOCU,0.105562
28,BEP,0.104757
183,GLUU,0.087755


In [111]:
best_stocks.to_csv("stock ranking.csv")

# Trend

In [ ]:
#Pull Only necessary columns
combined = combined.loc[:, ['Name', 'Symbol', '# of Shares']]

In [ ]:
Price_History = stockData(stocks['Ticker'], '2019-08-01', '2019-09-06')

In [ ]:
historical = pd.merge(combined, Price_History, how = 'outer', on = "Symbol", validate="many_to_many")


In [ ]:
historical['Value'] = historical['# of Shares'] * historical['Close']
historical.head()

In [ ]:
trend_data = historical.groupby(['Name', 'Day'])['Value'].sum().reset_index().sort_values(['Name', 'Day'])

In [ ]:
trend_data.head()

In [1]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode